<a href="https://colab.research.google.com/github/zyang63/Gate_design/blob/main/Distance_based_gating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Geometric Analysis in Google Colab


The code installs and imports several libraries for different purposes. The trimesh library is installed to work with triangular meshes, SimpleITK is installed for medical image analysis, scipy is installed for scientific computing, and bpy is installed for 3D modeling and rendering. The directory "geometry" is created to store the various stl files generated by the analysis. The result of the code is stored in the base directory and produces three "gltf" animations which can be downloaded and reviewed.

In [57]:
%%capture
try:
  import open3d as o3d
except ImportError:
  !pip install open3d
  import open3d as o3d
try:
  import SimpleITK as sitk
except ImportError:
  !pip install SimpleITK
  import SimpleITK as sitk
try:
  import bpy
except ImportError:
  !pip install bpy
  import bpy
try:
  from stl import mesh
except ImportError:
  !pip install numpy-stl
  from stl import mesh
try:
  import pyvista as pv
except ImportError:
  !pip install pyvista
  import pyvista as pv
import numpy as np
from scipy import ndimage
from skimage import measure
import os
import math

#Setup

In [58]:
import ipywidgets as widgets
from IPython.display import display
from google.colab import files
#@title #File Entry { display-mode: "form"}
#@markdown User can choose to upload the file to colab directly or select the google file upload button at the bottom of this form. Also choose the number of elements.

button_pressed = False  # Initialize the variable as False
filename = ""
button = widgets.Button(description="Google upload dialog")
output = widgets.Output()

def on_button_clicked(b):
    global button_pressed  # Access the global variable
    global filename
    with output:
        uploaded = files.upload()
        filename = list(uploaded.keys())[0]
        button_pressed = True  # Set the variable to True when the button is clicked

button.on_click(on_button_clicked)
display(button, output)

Button(description='Google upload dialog', style=ButtonStyle())

Output()

In [59]:
#@title #Element Count { display-mode: "form", run: "auto" }
element_count = 200 #@param {type:"slider", min:10, max:500, step:1}
if not button_pressed:
  filename = "/content/SA30053 3D MODEL_Casting.stl" #@param {type:"string"}
geometry = o3d.io.read_triangle_mesh(filename)
max_boundary_size = (geometry.get_max_bound()-geometry.get_min_bound()).max()
voxel_resolution = max_boundary_size/element_count
print("Element size is in units from stl file ",voxel_resolution, " per cell")
origin = geometry.get_min_bound()

Element size is in units from stl file  5.535  per cell


## Parameter setting

In [60]:
unit = 0.001 # stl size to m
injection_speed = 30.5 #m/s
injection_time = 0.5 #s

# Voxelizing the geometry


The following code performs voxelization and mesh conversion operations. It starts by voxelizing a mesh with a specified voxel size. Then, it creates a solid voxel representation by filling the voxel surface. The solid voxel data is padded with two layers of zeros. The padded voxel data is further processed using marching cubes algorithm to obtain a mesh representation. Finally, the resulting mesh is exported as an STL file named "marching_geometry.stl" in the geometry directory.

In [61]:
pcd = geometry.sample_points_uniformly(number_of_points=100000000)
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd,voxel_size=voxel_resolution)
voxels = voxel_grid.get_voxels()
indices = np.stack(list(vx.grid_index for vx in voxels))
del voxels, voxel_grid, pcd
max_indices = np.max(indices, axis=0)+1
dense_array = np.zeros(max_indices, dtype=np.bool8)
for idx in indices:
    dense_array[tuple(idx)] = 1
del indices
array_pad = np.pad(dense_array.astype(bool),((2,2)),'constant')
del dense_array
array_closing = ndimage.binary_closing(array_pad, structure=ndimage.generate_binary_structure(3, 1),
iterations=1, mask=None,  border_value=0, origin=0, brute_force=False)
img = sitk.GetImageFromArray(array_closing.astype(int))
seg = sitk.ConnectedComponent(img != img[0,0,0])
img_filled = sitk.BinaryFillhole(seg!=0)
array_filled = sitk.GetArrayFromImage(img_filled)
verts, faces, _, _ = measure.marching_cubes(array_filled)
vertices_original = np.asarray(verts.tolist())
faces_original = np.asarray(faces.tolist())

<ipython-input-61-87407ab34d32>:7: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  dense_array = np.zeros(max_indices, dtype=np.bool8)


# Morphological operations and Distance transform on mesh


The following code applies binary closing to a padded voxel data array and assigns the result to array_initial for further use. It then defines a function named core that performs operations to extract the core of a structure from an input array. The function is applied to array_initial, and the resulting array is assigned to array_core. The code further manipulates array_core by setting non-core elements to 0 and core elements to 1. Overall, these operations help identify and isolate the core of a structure within the voxel data.

In [62]:
array_initial = ndimage.binary_closing(array_filled, structure=ndimage.generate_binary_structure(3, 1), iterations=1, mask=None,  border_value=0, origin=0, brute_force=False)
img_initial = sitk.GetImageFromArray(array_initial.astype(int))

## Distance Field Calculation
The following code calculates the signed Maurer distance map from a binary voxel data representation. The distance map is computed using the SignedMaurerDistanceMap function from the SimpleITK library. The resulting distance map is then converted into a NumPy array. The code further determines the depth of the object by taking the negative minimum value from the distance map and converting it to an integer. This depth value represents the distance inside the object. Overall, these operations provide information about the spatial distribution and depth of the object in the voxel data.

In [63]:
img_dist = sitk.SignedMaurerDistanceMap(img_initial != 0, insideIsPositive=False, squaredDistance=False, useImageSpacing=False)
array_dist = sitk.GetArrayFromImage(img_dist)
depth = int(-array_dist.min())

## Watershed Segmentation

In [64]:
img_thresh = img_filled != 0
img_ws = sitk.MorphologicalWatershed(img_dist, markWatershedLine=True, level=1)
img_ws = sitk.Cast(img_ws, img_thresh.GetPixelID())
img_ws_mark = sitk.Mask(img_ws, img_thresh)
array_ws_mark = sitk.GetArrayFromImage(img_ws_mark)

## Parting Line Analysis

In [65]:
def sum_3d_array_along_height(input_array):
    height, rows, cols = input_array.shape
    result_array = np.sum(input_array, axis=2)

    return result_array

def find_edge(array):
    rows, cols = array.shape
    output = np.zeros((rows, cols), dtype=int)

    for i in range(rows):
        for j in range(cols):
            if array[i, j] != 0:
                if (i == 0 or array[i - 1, j] == 0) or \
                   (i == rows - 1 or array[i + 1, j] == 0) or \
                   (j == 0 or array[i, j - 1] == 0) or \
                   (j == cols - 1 or array[i, j + 1] == 0):
                    output[i, j] = 1

    return output

In [66]:
def find_overlap_3d_2d(arr_3d, arr_2d):
    overlap_arr = np.logical_and(arr_3d, arr_2d[:, :, np.newaxis]).astype(int)
    return overlap_arr

In [67]:
def extract_max_ones_layers(arr_3d):
    rows, cols, height = arr_3d.shape
    ones_count = np.sum(arr_3d, axis=(0, 1))
    max_ones_count = np.max(ones_count)
    layer_indices = np.where(ones_count == max_ones_count)[0]
    result_arr = np.zeros_like(arr_3d)
    result_arr[:, :, layer_indices] = arr_3d[:, :, layer_indices]

    return result_arr

In [68]:
S = sum_3d_array_along_height(array_filled)
B = find_edge(S)
N = find_overlap_3d_2d(array_filled, B)
C = extract_max_ones_layers(N)

T = find_edge(S)
P = np.zeros_like(T)
M = np.zeros_like(C)

while True:
    U = T - P
    V = find_overlap_3d_2d(N, U)
    L = extract_max_ones_layers(V)
    M = np.where(L == 1, 1, M)
    H = sum_3d_array_along_height(L)
    I = find_edge(H)
    P = np.where(I == 1, 1, P)

    if np.array_equal(P, T):
        break

In [69]:
layer_counts = np.sum(M, axis=(0, 1))

unique_counts, count_occurrences = np.unique(layer_counts, return_counts=True)
sorted_indices = np.argsort(count_occurrences)[::-1]

for i, count in enumerate(unique_counts[sorted_indices]):
    indices_to_replace = np.where(layer_counts == count)[0]

    for index in indices_to_replace:
        M[:, :, index][M[:, :, index] == 1] = i + 1

# Gate located


## count mesh number of segmentations

In [70]:
def calculate_counts(array_ws_mark):
    A = np.max(array_ws_mark)
    counts = np.zeros(A, dtype=int)

    for i in range(1, A + 1):
        counts[i - 1] = np.count_nonzero(array_ws_mark == i)

    return counts

## Find coordinates of the minimum value in distance field

In [71]:
def find_coordinates(array_dist):
    indices = np.nonzero(array_dist == array_dist.min())
    if indices[0].size == 0:
          return None
    return indices[0][0], indices[1][0], indices[2][0]

## Find the element closest to the minimum

In [72]:
def find_closest_nonzero_element(array_1, array_2, target_value):

    target_coords = np.argwhere(array_1 == target_value)
    if not target_coords.size:
        return None

    target_coords = target_coords[0]
    nonzero_mask = (array_2 != 0)
    distances = np.linalg.norm(np.argwhere(nonzero_mask) - target_coords, axis=1)
    min_distance_index = np.argmin(distances)
    closest_coords = tuple(np.argwhere(nonzero_mask)[min_distance_index])

    return closest_coords

## Find thick and  gate coordinates

In [73]:
def find_gate_coordinates(array_ws_mark, array_dist, voxel_resolution, origin):
    E_list = []
    X_list = []
    coordinates = []
    coordinates_min = []

    G = np.max(array_ws_mark) + 1

    for i in range(1, G):
        F_i = np.where(array_ws_mark != i, 0, array_ws_mark)
        T_i = np.where(F_i != i, 0, array_dist)
        M_i = T_i.min()
        gate_coord = find_closest_nonzero_element(T_i, M, M_i)
        coordinates_i = find_coordinates(T_i)
        coordinates.append(gate_coord)
        coordinates_min.append(coordinates_i)
        H_i = ((coordinates_i[0] - 2) * voxel_resolution - 1 + origin[0],
               (coordinates_i[1] - 2) * voxel_resolution + origin[1],
               (coordinates_i[2] - 2) * voxel_resolution + origin[2])
        X_list.append(H_i)
        E_i = ((gate_coord[0] - 2) * voxel_resolution + origin[0],
               (gate_coord[1] - 2) * voxel_resolution + origin[1],
               (gate_coord[2] - 2) * voxel_resolution + origin[2])
        E_list.append(E_i)

    return X_list, E_list, coordinates, coordinates_min

## Find number of segmentation

In [74]:
def get_values_and_display(array, coordinates_list):
    values_at_positions = []
    for x, y, z in coordinates_list:
        value = array[x, y, z]
        values_at_positions.append(value)
    return values_at_positions


## Find max thick of each segmentation

In [75]:
def get_values_and_distances(array_ws_mark, array_dist, coordinates_min_list):
    values_at_positions_min = [array_ws_mark[x, y, z] for x, y, z in coordinates_min_list]
    values_min = [array_dist[x, y, z] for x, y, z in coordinates_min_list]

    locations_min = []
    distance_min = []

    for i, position in enumerate(coordinates_min_list):
        locations_min.append(values_at_positions_min[i])
        distance_min.append(-values_min[i])

    return locations_min, distance_min


## Find max area of gate

In [76]:
def count_consecutive_ones(matrix, coordinates, locations, locations_min, distance_min):
    max_sizes = []

    for i, index in enumerate(coordinates):
        result_array = matrix[index[0], :, :]
        thickness = locations[i]
        seg_n = locations_min[i]
        dis_min = distance_min[i]

        row_count = [0] * result_array.shape[0]
        col_count = [0] * result_array.shape[1]

        for row_idx, row in enumerate(result_array):
            count = 0
            for element in row:
                if element == thickness:
                    count += 1
                    row_count[row_idx] = max(row_count[row_idx], count)
                else:
                    count = 0

        for col_idx in range(result_array.shape[1]):
            count = 0
            for row in result_array:
                if row[col_idx] == thickness:
                    count += 1
                    col_count[col_idx] = max(col_count[col_idx], count)
                else:
                    count = 0

        max_sizes.append((max(row_count), max(col_count), seg_n, dis_min))

    return max_sizes

## Merge segmentation

In [77]:
def euclidean_distance(point1, point2):
    return math.sqrt(sum((x - y) ** 2 for x, y in zip(point1, point2)))

def merge_segments(array_ws_mark, thick, max_size, X_list):
    merge_seg = []
    distances = []
    for element in thick:
        mask_1 = np.where(array_ws_mark == element, 1, 0)
        dilated_mask_1 = ndimage.binary_dilation(mask_1, structure=ndimage.generate_binary_structure(3, 2)).astype(mask_1.dtype)

        corresponding_elements = array_ws_mark[dilated_mask_1 == 1]
        filtered_array = corresponding_elements[(corresponding_elements != 0) & (corresponding_elements != element)]
        array_filtered_unique = np.unique(filtered_array)

        array_filtered_unique_updated = [x for x in array_filtered_unique if x not in thick]
        if not array_filtered_unique_updated:
            if array_filtered_unique.size > 0:
                array_filtered_unique_updated.append(array_filtered_unique[0])
            else:
                continue

        removed_elements = [x for x in array_filtered_unique if x in thick]

        thick = np.array([element[2] for element in max_size])
        matching_indices = np.where(np.isin(thick, array_filtered_unique_updated))
        distance_arrays = [X_list[i] for i in matching_indices[0]]
        output_arrays = [max_size[i] for i in matching_indices[0]]

        for point in distance_arrays:
          distance = euclidean_distance(point, X_list[int(element)])
          distances.append(distance)
        distance_list = [(tup[0], tup[1], tup[2], distances[i]) for i, tup in enumerate(output_arrays)]
        output_arrays_sorted = sorted(distance_list, key=lambda x: x[3] ,reverse=False)

        largest_d = min(output_arrays_sorted, key=lambda x: x[3])[3]
        tuples_with_largest_d = [t for t in output_arrays_sorted if t[3] == largest_d]
        products = [(t[0] * t[1], t[2]) for t in tuples_with_largest_d]
        larger_product = max(products, key=lambda x: x[0])
        merge_seg.append(larger_product[1])

    return merge_seg

In [78]:
def replace_elements(array_3d, array_1, array_2):

    new_array_2 = np.copy(array_2)

    while True:
        replaced = False
        for i, element in enumerate(new_array_2):
            if np.any(array_1 == element):
                indices_in_array_1 = np.where(array_1 == element)[0]
                for index_in_array_1 in indices_in_array_1:
                    if new_array_2[i] != array_2[index_in_array_1]:
                        new_array_2[i] = array_2[index_in_array_1]
                        replaced = True
        if not replaced:
            break

    result_array = np.copy(array_3d)
    for value_to_replace, replacement_value in zip(array_1, new_array_2):
        result_array[result_array == value_to_replace] = replacement_value

    return result_array

## Calculate the size of the merged gate

In [79]:
def main(array_ws_mark, array_dist, voxel_resolution, origin, M, injection_speed, injection_time, unit):
    global last_max_size, last_result

    counts = calculate_counts(array_ws_mark)
    gate_size = np.array(((counts * ((voxel_resolution * unit) ** 3)) / (injection_speed * injection_time)))
    X_list, E_list, coordinates, coordinates_min = find_gate_coordinates(array_ws_mark, array_dist, voxel_resolution, origin)
    locations = get_values_and_display(M, coordinates)
    locations_min, distance_min = get_values_and_distances(array_ws_mark, array_dist, coordinates_min)
    max_size_O = count_consecutive_ones(M, coordinates, locations, locations_min, distance_min)

    result = array_ws_mark
    merge_seg_previous = None

    while True:
        counts = calculate_counts(result)
        gate_size = np.array(((counts * ((voxel_resolution * unit) ** 3)) / (injection_speed * injection_time)))
        X_list, E_list, coordinates, coordinates_min = find_gate_coordinates(result, array_dist, voxel_resolution, origin)
        locations = get_values_and_display(M, coordinates)
        locations_min, distance_min = get_values_and_distances(result, array_dist, coordinates_min)
        max_size = [(tup[0], tup[1], tup[2], distance_min[i]) for i, tup in enumerate(max_size_O)]
        count = counts.tolist()

        for i in range(len(count)):
            if count[i] == 0:
                max_size[i] = (0, 0, 0, 0)

        Q = gate_size / (np.array([ms[0] for ms in max_size]) * (voxel_resolution * unit) ** 2)
        if not Q.any():
            print("Q is empty. Exiting loop.")
            break

        found = any(Q[i] > max(max_size[i][:2]) * 1.15 for i in range(len(max_size)))
        if found:
            for i in range(len(max_size)):
                smaller_element = min(max_size[i][:2])
                larger_element = max(max_size[i][:2])
                if Q[i] > larger_element * 1.15:
                    filtered_max_size = [element for element, q_value in zip(max_size, Q) if q_value > max(element[:2])]
                    filtered_max_size_sorted = sorted(filtered_max_size, key=lambda x: x[3])
                    filtered_max_size_sorted_np = np.array(filtered_max_size_sorted)
                    third_elements = filtered_max_size_sorted_np[:, 2]
                    third_elements_list = third_elements.tolist()
                    merge_seg_current = merge_segments(result, third_elements, max_size, X_list)
                    result = replace_elements(result, third_elements_list, merge_seg_current)

            last_max_size = max_size

            if merge_seg_current == merge_seg_previous:
                break

            merge_seg_previous = merge_seg_current
        else:
            last_result = result
            last_max_size = max_size
            break

    last_result = result
    print("Last max_size:", last_max_size)
main(array_ws_mark, array_dist, voxel_resolution, origin, M, injection_speed, injection_time, unit)

Last max_size: [(1, 119, 1, 9.433981), (1, 119, 2, 9.433981)]


In [80]:
channel = max_boundary_size * unit

cuboid_sizes_list = []

for i in range(len(last_max_size)):
    smaller_element = min(last_max_size[i][:2])
    larger_element = max(last_max_size[i][:2])
    count_merge = np.count_nonzero(last_result == i + 1)

    gate_size_merge = np.array(((count_merge * ((voxel_resolution * unit) ** 3)) / (injection_speed * injection_time)))
    Q = gate_size_merge / ((voxel_resolution * unit) ** 2)
    W = (gate_size_merge / ((voxel_resolution * unit) ** 2)) ** 0.5

    if W <= smaller_element:
        max_index = last_max_size[i][:2].index(max(last_max_size[i][:2]))
        last_max_size[i] = tuple(W if j in [0, 1] and j == max_index else val for j, val in enumerate(last_max_size[i]))
    elif W > smaller_element and (Q <= larger_element).all():
        max_index = last_max_size[i][:2].index(max(last_max_size[i][:2]))
        last_max_size[i] = tuple(Q if j in [0, 1] and j == max_index else val for j, val in enumerate(last_max_size[i]))

cuboid_sizes = [(item[0] * voxel_resolution, item[1] * voxel_resolution,
                  (0 if item[0] == 0 and item[1] == 0 else channel) * voxel_resolution)
                 for item in last_max_size]

cuboid_sizes_list.append(cuboid_sizes)  # Appending cuboid sizes to the list

print(cuboid_sizes_list)


[[(5.535, 512.8463886344261, 6.127245), (5.535, 512.8463886344261, 6.127245)]]


# Gate generated

In [81]:
def calculate_rotation_matrix(v, w):
    cos_theta = np.dot(v, w) / (np.linalg.norm(v) * np.linalg.norm(w))
    sin_theta = np.sqrt(1 - cos_theta**2)
    theta = np.arccos(cos_theta)

    u = np.cross(v, w) / np.linalg.norm(np.cross(v, w))
    R = np.array([
        [cos_theta + u[0]**2 * (1 - cos_theta), u[0]*u[1]*(1 - cos_theta) - u[2]*sin_theta,
         u[0]*u[2]*(1 - cos_theta) + u[1]*sin_theta],
        [u[1]*u[0]*(1 - cos_theta) + u[2]*sin_theta, cos_theta + u[1]**2 * (1 - cos_theta),
         u[1]*u[2]*(1 - cos_theta) - u[0]*sin_theta],
        [u[2]*u[0]*(1 - cos_theta) - u[1]*sin_theta,
         u[2]*u[1]*(1 - cos_theta) + u[0]*sin_theta, cos_theta + u[2]**2 * (1 - cos_theta)]])

    return R

def create_cuboid(cuboid_size):
    vertices = np.array([
        [-0.5, -0.5, 0] * cuboid_size,[0.5, -0.5, 0] * cuboid_size,[0.5, 0.5, 0] * cuboid_size,
         [-0.5, 0.5, 0] * cuboid_size,[-0.5, -0.5, 2] * cuboid_size,[0.5, -0.5, 2] * cuboid_size,
          [0.5, 0.5, 2] * cuboid_size,[-0.5, 0.5, 2] * cuboid_size])
    return vertices

all_cuboid_meshes = []
X_list, E_list, coordinates, coordinates_min = find_gate_coordinates(array_ws_mark, array_dist, voxel_resolution, origin)

for point_A, point_B, cuboid_size  in zip(X_list, E_list, cuboid_sizes):
    point_A = np.array(point_A)
    point_B = np.array(point_B)
    cuboid_size = np.array(cuboid_size)
    original_direction = np.array([0, 0, 1])
    target_direction = (point_B - point_A) * np.array([1, 1, 0])
    rotation_matrix = calculate_rotation_matrix(original_direction, target_direction)
    cuboid_vertices = create_cuboid(cuboid_size)
    rotated_vertices = point_B + np.dot(cuboid_vertices, rotation_matrix.T)
    faces = np.array([
        [0, 1, 2],[0, 2, 3],[4, 6, 5],[4, 7, 6],[0, 4, 5],
        [0, 5, 1],[2, 6, 7],[2, 7, 3],[0, 3, 7],[0, 7, 4],
        [1, 5, 6],[1, 6, 2]])

    cuboid_mesh = mesh.Mesh(np.zeros(len(faces), dtype=mesh.Mesh.dtype))
    for i, face in enumerate(faces):
        cuboid_mesh.vectors[i] = rotated_vertices[face]
    all_cuboid_meshes.append(cuboid_mesh)
combined_mesh = mesh.Mesh(np.concatenate([mesh.data for mesh in all_cuboid_meshes]))
combined_mesh.save('all_gate.stl')

In [82]:
bpy.ops.wm.read_factory_settings(use_empty=True)
bpy.ops.import_mesh.stl(filepath=filename)
obj = bpy.context.object
obj.select_set(True)
bpy.context.view_layer.objects.active = obj
obj.color = (0,0,1,1)
mat = bpy.data.materials.new("0")
mat.use_nodes = True
principled = mat.node_tree.nodes["Principled BSDF"]
principled.inputs["Base Color"].default_value = (1,1,1,1)
principled.inputs["Alpha"].default_value = 0.7
obj.data.materials.append(mat)
bpy.context.object.active_material.blend_method = "BLEND"
bpy.ops.import_mesh.stl(filepath="//content//all_gate.stl")
obj = bpy.context.object
obj.color = (0,0,1,1)
mat = bpy.data.materials.new("0")
mat.use_nodes = True
principled = mat.node_tree.nodes["Principled BSDF"]
principled.inputs["Base Color"].default_value = (0,1,0,1)
obj.data.materials.append(mat)
bpy.ops.export_scene.gltf(filepath="//content//colored_gating.glb")

Import finished in 0.0052 sec.
Import finished in 0.0003 sec.
02:11:42 | ERROR: Draco mesh compression is not available because library could not be found at /content/4.0/python/lib/python3.10/site-packages/libextern_draco.so
02:11:42 | INFO: Starting glTF 2.0 export
02:11:42 | INFO: Extracting primitive: SA30053 3D MODEL_Casting
02:11:42 | INFO: Primitives created: 1
02:11:42 | INFO: Extracting primitive: all_gate
02:11:42 | INFO: Primitives created: 1
02:11:42 | INFO: Finished glTF 2.0 export in 0.022060871124267578 s



{'FINISHED'}